In [1]:
%matplotlib inline

In [2]:
# Import training data
import matplotlib.pyplot as plt
import csv
import numpy as np

np.random.seed(0)

def import_csv(csvfile):
    """
    Imports the CSV file as an array of lines
    
    :param csvfile: The path of the CSV file
    :return lines: The lines of the CSV file 
    """
    lines = []
    with open(log_path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)

    return lines

data_path = './data/'
img_path = data_path + 'IMG/'
log_path = data_path + 'driving_log.csv'

# Store the lines of the CSV file in an array
lines = import_csv(log_path)

# Remove the first line of the array
lines.pop(0)

# Transform to a relative path to the images
for line in lines:
    line[0] = line[0].split('/')[-1]
    line[1] = line[1].split('/')[-1]
    line[2] = line[2].split('/')[-1]
    
# Make lines a numpy array
lines = np.asarray(lines)
    
# Test to make sure we read that properly
print(lines[0])
print(lines.shape)

['center_2016_12_01_13_30_48_287.jpg' 'left_2016_12_01_13_30_48_287.jpg'
 'right_2016_12_01_13_30_48_287.jpg' ' 0' ' 0' ' 0' ' 22.14829']
(8036, 7)


In [3]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split

X = lines[:,0:3]
y = lines[:,3]

# Convert steering values to float
y = y.astype(np.float32)

test_size = 0.2
random_state = 0

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Confirmation
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

print(X_train[0])
print(y_train[0])

(6428, 3)
(6428,)
(1608, 3)
(1608,)
['center_2016_12_01_13_43_40_097.jpg' 'left_2016_12_01_13_43_40_097.jpg'
 'right_2016_12_01_13_43_40_097.jpg']
0.0


In [4]:
# Build the model
from keras.models import Sequential
from keras.layers import Lambda, Dropout, Conv2D, Dense, MaxPooling2D, Flatten

keep_prob = 0.5
height = 66
width = 200
num_channels = 3

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(height, width, num_channels)))
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(keep_prob))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))

model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 47, 36)    21636       convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 5, 22, 48)     43248       convolution2d_2[0][0]            
___________________________________________________________________________________________

In [5]:
# Functions to process the images
import matplotlib.image as mpimg
import os
import cv2

def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))


def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom)
    """
    return image[60:-25, :, :] # remove the sky and the car front


def resize(image):
    """
    Resize the image to the input shape used by the network model
    """
    return cv2.resize(image, (width, height), cv2.INTER_AREA)


def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)


def preprocess(image):
    """
    Combine all preprocess functions into one
    """
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    
    return image


def choose_image(data_dir, center, left, right, steering_angle):
    """
    Randomly choose an image from the center, left or right, and adjust
    the steering angle.
    """
    choice = np.random.choice(3)
    
    if choice == 0:
        return load_image(data_dir, left), steering_angle + 0.2
    elif choice == 1:
        return load_image(data_dir, right), steering_angle - 0.2
    
    return load_image(data_dir, center), steering_angle


def random_flip(image, steering_angle):
    """
    Randomly flipt the image left <-> right, and adjust the steering angle.
    """
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
        
    return image, steering_angle


def random_translate(image, steering_angle, range_x, range_y):
    """
    Randomly shift the image virtially and horizontally (translation).
    """
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    
    return image, steering_angle


def random_shadow(image):
    """
    Generates and adds random shadow
    """
    # (x1, y1) and (x2, y2) forms a line
    # xm, ym gives all the locations of the image
    x1, y1 = width * np.random.rand(), 0
    x2, y2 = width * np.random.rand(), height
    xm, ym = np.mgrid[0:height, 0:width]

    # mathematically speaking, we want to set 1 below the line and zero otherwise
    # Our coordinate is up side down.  So, the above the line: 
    # (ym-y1)/(xm-x1) > (y2-y1)/(x2-x1)
    # as x2 == x1 causes zero-division problem, we'll write it in the below form:
    # (ym-y1)*(x2-x1) - (y2-y1)*(xm-x1) > 0
    mask = np.zeros_like(image[:, :, 1])
    mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1

    # choose which side should have shadow and adjust saturation
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    # adjust Saturation in HLS(Hue, Light, Saturation)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)


def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    """
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


def augment_image(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    """
    Generate an augumented image and adjust steering angle.
    (The steering angle is associated with the center image)
    """
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = random_shadow(image)
    image = random_brightness(image)
    
    return image, steering_angle

In [6]:
# Build the generator
def generator(data_dir, image_paths, steer_angles, batch_size, is_training):
    """
    Generator to process a certain portion of the model at a time
    
    :param data_dir: The data directory
    :param image_paths: The paths to the images
    :param steer_angles: The steering angles
    :param batch_size: The batch size
    :param is_training: Whether this is training data (True) or validation data (False)
    """
    images = np.empty([batch_size, height, width, num_channels])
    steering = np.empty(batch_size)
    
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steer_angles[index]
            
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augment_image(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
                
            image = preprocess(image)
            
            images[i] = image
            steering[i] = steering_angle
            
            # debug
            #print(image.shape)
            
            i += 1
            if i == batch_size:
                break
                
        yield images, steering

In [7]:
# Train the model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

batch_size = 40
num_epochs = 10
samples_per_epoch = 20000
learn_rate = 0.0001

checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

model.compile(loss='mean_squared_error', optimizer=Adam(lr=learn_rate))

model.fit_generator(generator(img_path, X_train, y_train, batch_size, True),
                    samples_per_epoch,
                    num_epochs,
                    max_q_size=1,
                    validation_data=generator(img_path, X_valid, y_valid, batch_size, False),
                    nb_val_samples=len(X_valid),
                    callbacks=[checkpoint],
                    verbose=1)

Epoch 1/10


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:101: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 160 but corresponding boolean dimension is 66


20000/20000 [==============================] - 154s - loss: 0.0339 - val_loss: 0.0115